## Utils

In [4]:
# %%writefile /content/Vis_Odometry/vis_odom_utils.py
import numpy as np
import cv2

def umeyama_alignment(x, y, with_scale=True):
    """
    Computes the least squares solution parameters of an Sim(m) matrix
    that minimizes the distance between a set of registered points.
    Umeyama, Shinji: Least-squares estimation of transformation parameters
                     between two point patterns. IEEE PAMI, 1991
    :param x: mxn matrix of points, m = dimension, n = nr. of data points
    :param y: mxn matrix of points, m = dimension, n = nr. of data points
    :param with_scale: set to True to align also the scale (default: 1.0 scale)
    :return: r, t, c - rotation matrix, translation vector and scale factor
    """
    if x.shape != y.shape:
        assert False, "x.shape not equal to y.shape"

    # m = dimension, n = nr. of data points
    m, n = x.shape

    # means, eq. 34 and 35
    mean_x = x.mean(axis=1)
    mean_y = y.mean(axis=1)

    # variance, eq. 36
    # "transpose" for column subtraction
    sigma_x = 1.0 / n * (np.linalg.norm(x - mean_x[:, np.newaxis])**2)

    # covariance matrix, eq. 38
    outer_sum = np.zeros((m, m))
    for i in range(n):
        outer_sum += np.outer((y[:, i] - mean_y), (x[:, i] - mean_x))
    cov_xy = np.multiply(1.0 / n, outer_sum)

    # SVD (text betw. eq. 38 and 39)
    u, d, v = np.linalg.svd(cov_xy)

    # S matrix, eq. 43
    s = np.eye(m)
    if np.linalg.det(u) * np.linalg.det(v) < 0.0:
        # Ensure a RHS coordinate system (Kabsch algorithm).
        s[m - 1, m - 1] = -1

    # rotation, eq. 40
    r = u.dot(s).dot(v)

    # scale & translation, eq. 42 and 41
    c = 1 / sigma_x * np.trace(np.diag(d).dot(s)) if with_scale else 1.0
    t = mean_y - np.multiply(c, r.dot(mean_x))

    return r, t, c


def unprojection_kp(kp, kp_depth, cam_intrinsics):
    """Convert kp to XYZ
    Args:
        kp (Nx2 array): [x, y] keypoints
        kp_depth (Nx2 array): keypoint depth
        cam_intrinsics (Intrinsics): camera intrinsics
    Returns:
        XYZ (Nx3): 3D coordinates
    """
    N = kp.shape[0]
    # initialize regular grid
    XYZ = np.ones((N, 3, 1))
    XYZ[:, :2, 0] = kp
    
    inv_K = np.ones((1, 3, 3))
    inv_K[0] = np.linalg.inv(cam_intrinsics)
    inv_K = np.repeat(inv_K, N, axis=0)

    XYZ = np.matmul(inv_K, XYZ)[:, :, 0]
    XYZ[:, 0] = XYZ[:, 0] * kp_depth
    XYZ[:, 1] = XYZ[:, 1] * kp_depth
    XYZ[:, 2] = XYZ[:, 2] * kp_depth
    return XYZ

def normalize_kp(kp, cam_intr):
    kp_norm = kp.copy()
    kp_norm[:, 0] = \
        (kp[:, 0] - cam_intr[0,2]) / cam_intr[0,0]
    kp_norm[:, 1] = \
        (kp[:, 1] - cam_intr[1,2]) / cam_intr[1,1]

    kp1_3D = np.ones((3, kp_norm.shape[0]))
    kp1_3D[0], kp1_3D[1] = kp_norm[:, 0].copy(), kp_norm[:, 1].copy()

    return kp1_3D

def triangulation(kp1, kp2, T_1w, T_2w, cam_intr):
    """Triangulation to get 3D points
    Args:
        kp1 (Nx2): keypoint in view 1 (normalized)
        kp2 (Nx2): keypoints in view 2 (normalized)
        T_1w (4x4): pose of view 1 w.r.t  i.e. T_1w (from w to 1)
        T_2w (4x4): pose of view 2 w.r.t world, i.e. T_2w (from w to 2)
    Returns:
        X (3xN): 3D coordinates of the keypoints w.r.t world coordinate
        X1 (3xN): 3D coordinates of the keypoints w.r.t view1 coordinate
        X2 (3xN): 3D coordinates of the keypoints w.r.t view2 coordinate
    """
    
    kp1_norm = kp1.copy()
    kp2_norm = kp2.copy()

    kp1_norm[:, 0] = \
        (kp1[:, 0] - cam_intr[0,2]) / cam_intr[0,0]
    kp1_norm[:, 1] = \
        (kp1[:, 1] - cam_intr[1,2]) / cam_intr[1,1]
    kp2_norm[:, 0] = \
        (kp2[:, 0] - cam_intr[0,2]) / cam_intr[0,0]
    kp2_norm[:, 1] = \
        (kp2[:, 1] - cam_intr[1,2]) / cam_intr[1,1]

    kp1_3D = np.ones((3, kp1_norm.shape[0]))
    kp2_3D = np.ones((3, kp2_norm.shape[0]))
    kp1_3D[0], kp1_3D[1] = kp1_norm[:, 0].copy(), kp1_norm[:, 1].copy()
    kp2_3D[0], kp2_3D[1] = kp2_norm[:, 0].copy(), kp2_norm[:, 1].copy()

    X = cv2.triangulatePoints(T_1w[:3], T_2w[:3], kp1_3D[:2], kp2_3D[:2])
    X /= X[3]
    X1 = T_1w[:3] @ X
    X2 = T_2w[:3] @ X
    return X[:3].T, X1, X2


def transform_points(points_3D, pose):
  points_4D = np.c_[ points_3D,  np.ones(len(points_3D)) ].T
  transformed_points_4D = pose @ points_4D
  points_3D = transformed_points_4D[:3].T
  return points_3D



def find_scale(f13D, f23D):

    valid_1 = f13D[...,2] > 0
    valid_2 = f23D[...,2] > 0
    valid = valid_1 & valid_2
    f13D = f13D[valid]
    f23D = f23D[valid]
    indices = np.random.choice(np.arange(0,len(f23D)), size=(5 * len(f23D),2),replace=True)
    indices = indices[indices[...,0]!=indices[...,1]]
    num = np.linalg.norm(f13D[indices[...,0]] - f13D[indices[...,1]], axis=1).reshape((len(indices),1))
    den = np.linalg.norm(f23D[indices[...,0]] - f23D[indices[...,1]], axis=1).reshape((len(indices),1))
    ransac = linear_model.RANSACRegressor(
                base_estimator=linear_model.LinearRegression(
                    fit_intercept=False),
                min_samples=2,
                max_trials=100,
                stop_probability=0.99,
                residual_threshold=1.0
                                        )
    ransac.fit(
            num,
            den
            )
    
    scale = ransac.estimator_.coef_[0, 0]
    return scale
    return np.median(den/num)

def find_scale_depthonly(real_depth, unscaled_depth):

    valid_1 = real_depth > 0
    valid_2 = unscaled_depth > 0
    valid = valid_1 & valid_2
    real_depth = real_depth[valid]
    unscaled_depth = unscaled_depth[valid]

    # residuals = 

    # Estimate scale (ransac)
    if valid.sum() > 0: #self.cfg.translation_scale.ransac.min_samples:
        # RANSAC scaling solver
        ransac = linear_model.RANSACRegressor(
                    base_estimator=linear_model.LinearRegression(
                        fit_intercept=False),
                    min_samples=3,
                    max_trials=100,
                    stop_probability=0.99,
                    residual_threshold=1
                    )
        ransac.fit(
                f13D.reshape(-1, 1),
                f23D.reshape(-1, 1)
                )
        scale = ransac.estimator_.coef_[0, 0]
    else:
        scale = -1

    return scale

# Taken from  DF-VO : https://github.com/Huangying-Zhan/DF-VO
import numpy as np
class SE3():
    """SE3 object consists rotation and translation components
    Attributes:
        pose (4x4 numpy array): camera pose
        inv_pose (4x4 numpy array): inverse camera pose
        R (3x3 numpy array): Rotation component
        t (3x1 numpy array): translation component,
    """
    def __init__(self, np_arr=None):
        if np_arr is None:
            self._pose = np.eye(4)
        else:
            self._pose = np_arr

    @property
    def pose(self):
        """ pose (4x4 numpy array): camera pose """
        return self._pose

    @pose.setter
    def pose(self, value):
        self._pose = value

    @property
    def inv_pose(self):
        """ inv_pose (4x4 numpy array): inverse camera pose """
        return np.linalg.inv(self._pose)

    @inv_pose.setter
    def inv_pose(self, value):
        self._pose = np.linalg.inv(value)

    @property
    def R(self):
        return self._pose[:3, :3]

    @R.setter
    def R(self, value):
        self._pose[:3, :3] = value

    @property
    def t(self):
        return self._pose[:3, 3:]

    @t.setter
    def t(self, value):
        self._pose[:3, 3:] = value


import numpy as np
# from sim3 import SE3
class FramePair:
    def __init__(self, f1, f2, matches_no, left_kp, right_kp, frame1_idx = None,  cheirality_pts_ct=0, inlier_pts_ct=0, pose=SE3()):
        self.frame1 = f1
        self.frame2 = f2
        self.left_kp = left_kp
        self.right_kp = right_kp
        self.cheirality_pts_ct = cheirality_pts_ct
        self.inlier_pts_ct = inlier_pts_ct
        self.pose = pose
        self.avg_optical_flow = 0
        self.matches_no = matches_no
        self.ess_mat = None
        self.frame_index = frame1_idx


    def getpose(self):
        # return self.pose
        pos = SE3()
        pos.t = self.pose.t.copy()
        pos.R = self.pose.R.copy()
        return pos

    def getkeypts(self):
        return (self.left_kp, self.right_kp)
    
    def getchecks(self):
        return (self.cheirality_pts_ct, self.inlier_pts_ct)

class Frame:
    def __init__(self, id, img, kps, desc, fil, pose = SE3(), seg_img = None, depth = np.zeros(1), image_arr = None, ):
        self.id = id
        self.image = img
        self.keypoints = kps
        self.descriptors = desc
        self.filename = fil
        self.depth = depth
        self.pose = pose
        self.image_arr = image_arr
        self.glob_pose = None
        self.tracked_kps = None
        self.global_pose = None
        self.kps_index = np.arange(len(kps))
        self.seg_img = seg_img

    
    def getitems(self):
        return (self.image, self.keypoints, self.descriptors, self.filename)

    def get_kp_desc(self):
        return (self.keypoints, self.descriptors)

    def get_image(self):
        return self.image

    def get_file(self):
        return self.filename


In [5]:
# %%writefile /content/homography_computation/auto_park_utils.py
import torch
import torchvision.transforms as transforms
import cv2 
from google.colab.patches import cv2_imshow
from PIL import Image
import numpy as np 
import glob
import sys
sys.path.insert(1, 'ShelfNet18_realtime/')
from evaluate import MscEval
from shelfnet import ShelfNet

class auto_park_vision():
    def __init__(self,weights_path):

        self.n_classes = 19
        self.eval_define() #Define Object of class MscEval

    def forward_pass(self,frame=None,img_path=None):

        if(img_path != None):
            img = Image.open(img_path)
        else:
            img = frame 
        orig_img = np.array(img)
        # orig_img = cv2.imread(img_path)
        # cv2_imshow(img)
        #Preprocess Image
        to_tensor = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])
        img = to_tensor(img)
        

        _, H, W = img.shape
        # print("H,W:",H,W)
        #Change image size to the form NCHW from CHW
        img = img.unsqueeze(0)

        probs = torch.zeros((1, self.n_classes, H, W))
        probs.requires_grad = False
        img = img.cuda()        

        # for sc in self.scales:
        prob = self.evaluator.scale_crop_eval(img, scale=1.0) #prob.type torch.cuda.FloatTensor
        prob = prob.detach().cpu()
        prob = prob.data.numpy()
        preds = np.argmax(prob, axis=1) #preds.dtype int64

        #Changed 
        preds = preds.squeeze().astype(np.uint8)
        preds[preds == 0] = 255
        preds = preds.astype(np.uint8)
        return preds


    def eval_define(self):

        n_classes = self.n_classes
        net = ShelfNet(n_classes=n_classes)

        net.load_state_dict(torch.load(weights_path))
        net.cuda()
        net.eval()
        self.evaluator = MscEval(net, dataloader=None, scales=[1.0],flip=False)



In [6]:
# %%writefile /content/homography_computation/main.py
import glob
import numpy as np
import cv2 
import os
from google.colab.patches import cv2_imshow
import time
from statistics import mean
import pickle

def world_2d(points_ls):
    '''
    Convert a list of 3d points to corresponding image points using homography
    '''
    points_2d_ls = []
    for point in points_ls:
        point_3d = np.asarray(point)
        point_3d = point_3d.reshape(3,1)
        # print("point_3d",point_3d)
        point_2d = np.dot(H,point_3d)
        point_2d = point_2d // point_2d[2,0]
        points_2d_ls.append(point_2d)
        # print("point_2d",point_2d)
    return points_2d_ls

def project_points(img,points_2d_ls):
    '''
    Draw a bounding rectangle on the image using the computed homography
    '''
    # cv2_imshow(img)
    # for point_2d in points_2d_ls:
    #     cv2.circle(img,(point_2d[0],point_2d[1]), 5, (0,0,255), -1)
        
    #For visualization
    N = len(points_2d_ls)
    for i in range(0,N):
        cv2.line(img,(points_2d_ls[i%N][0],points_2d_ls[i%N][1]),\
                 (points_2d_ls[(i+1)%N][0],points_2d_ls[(i+1)%N][1]),\
                 (0,0,255),4)
    #Image with points drawn on it
    cv2_imshow(img)

def pot_parking_spot(orig_img,inf_img,points_2d_ls):
    '''
    Function to detect potential parking spot
    '''
    #TODO: Optimize the code
    xcoords_ls = []
    ycoords_ls = []
    for point in points_2d_ls:
        xcoords_ls.append(point[0][0])
        ycoords_ls.append(point[1][0])
    
    #Line equations of the top and bottom line
    coeff_top = np.polyfit(xcoords_ls[0:2],ycoords_ls[0:2],1)
    line_top = np.poly1d(coeff_top)

    coeff_bottom = np.polyfit(xcoords_ls[2:4],ycoords_ls[2:4],1)
    line_bottom = np.poly1d(coeff_bottom)
    # print("line_bottom",line_bottom)
    # print("line_top",line_top)
    flag_top = 0
    flag_bottom = 0
    #Points for potential parking spot
    pt_tl = []
    pt_tr = []
    pt_bl = []
    pt_br = []
    for x in range(0,inf_img.shape[1]):
        if(inf_img[int(line_top(x)),x] == 255 and flag_top == 0):
            pt_tl = [int(line_top(x)),x]
            pt_tr = [int(line_top(x+200)),int(x+200)]
            # cv2.circle(orig_img,(pt_tl[1],pt_tl[0]), 5, (0,0,255), -1)
            # cv2.circle(orig_img,(pt_tr[1],pt_tr[0]), 5, (0,0,255), -1)
            # cv2_imshow(orig_img)
            flag_top = 1

        if(inf_img[int(line_bottom(x)),x] == 255 and flag_bottom == 0):
            pt_bl = [int(line_bottom(x)),x]
            pt_br = [int(line_bottom(x+200)),int(x+200)]
            # cv2.circle(orig_img,(pt_bl[1],pt_bl[0]), 5, (0,0,255), -1)
            # cv2.circle(orig_img,(pt_br[1],pt_br[0]), 5, (0,0,255), -1)
            # cv2_imshow(orig_img)
            flag_bottom = 1
        
        if(flag_top == 1 and flag_bottom ==1):
            # cv2_imshow(orig_img)
            break

    if(flag_top == 1 and flag_bottom == 1):
        cv2.line(orig_img,(pt_tl[1],pt_tl[0]),(pt_tr[1],pt_tr[0]),(0,0,255),2)
        cv2.line(orig_img,(pt_tr[1],pt_tr[0]),(pt_br[1],pt_br[0]),(0,0,255),2)
        cv2.line(orig_img,(pt_br[1],pt_br[0]),(pt_bl[1],pt_bl[0]),(0,0,255),2)
        cv2.line(orig_img,(pt_bl[1],pt_bl[0]),(pt_tl[1],pt_tl[0]),(0,0,255),2)
    
    pt_ls = [pt_bl,pt_br,pt_tr,pt_tl]
    return orig_img,pt_ls
    
    
def chk_cnts(pt_ls):
    '''
    Check if contour pt_ls contains invalid entries
    '''
    for pt in pt_ls:
        if not pt:
            return False
    
    return True
def ret_line_eq(pt1,pt2):
    '''
    Returns m1,c1 given 2 points
    '''
    points = [pt1,pt2]
    x_coords, y_coords = zip(*points)
    A = np.vstack([x_coords,np.ones(len(x_coords))]).T
    m1, c1 = np.linalg.lstsq(A, y_coords)[0] #TODO: Improve this, it computes the least square solution
    return m1,c1


def find_midpoint(pt_ls):
    '''
    Find the midpoint given 4 corners of the contour
    '''
    m1,c1 = ret_line_eq(pt_ls[0],pt_ls[2])
    m2,c2 = ret_line_eq(pt_ls[1],pt_ls[3])

    #Solve the 2 eqns to obtain the midpoint
    A = np.array([[-m1,1],
                 [-m2,1]],dtype=np.float64)
    B = np.array([c1,c2])
    midpoint = np.linalg.inv(A).dot(B)
    #Inverse homography matrix
    inv_h = np.linalg.inv(H)
    #Conversion from Numpy coordinates to OpenCV coordinates
    #TODO: Optimize this computation
    midpoint_homography = np.copy(midpoint).reshape(2,1)
    midpoint_homography[0,0] = midpoint[1]
    midpoint_homography[1,0] = midpoint[0]
    midpoint_homography = np.append(midpoint_homography,1).reshape(-1,1)
    # midpoint_homography.append(midpoint[0])
    # midpoint_homography.append(1)
    world_midpoint = np.dot(inv_h,midpoint_homography)
    world_midpoint = world_midpoint / world_midpoint[2,0]
    return world_midpoint , midpoint
    



## Specify Feature Extractor

In [ ]:
from R2D2 import *

# VO Code

In [26]:
# Version - Shrutheesh


# %%writefile /content/Vis_Odometry/main_cell.py
# Parking Spot
import cv2
import numpy as np
import time
import copy
import pickle
from tqdm import tqdm
import sys
import glob
import re
import shutil
from sklearn import linear_model
import os
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from scipy.spatial.transform import Rotation as sciPyR

np.random.seed(4123)

listlength = 1

def multidim_intersect(arr1, arr2):
    #DOUBT: Why are we performing arr1_view and arr2_view?
    # arr1_view = arr1.view([('',arr1.dtype)]*arr1.shape[1])
    # arr2_view = arr2.view([('',arr2.dtype)]*arr2.shape[1])
    arr1 = arr1.cpu().numpy()
    arr2 = arr2.cpu().numpy()
    arr1_view = arr1.view([('',arr1.dtype)]*arr1.shape[1])
    arr2_view = arr2.view([('',arr2.dtype)]*arr2.shape[1])
    intersected, ind1, ind2= np.intersect1d(arr1_view, arr2_view, return_indices = True)
    # print("intersected",intersected)
    # print("ind1",ind1)
    # print("ind2",ind2)
    return (intersected,ind1,ind2)
    # return (intersected.view(arr1.dtype).reshape(-1, arr1.shape[1]), ind2)ind1,  

def append_to_list(l, ele, listlen= listlength):
    l.append(ele)
    return l[-listlen:]

class VisualOdometry:
        def __init__(self, cam_intr, homo_matrix):

 
                self.cam_intr = cam_intr.copy()            

                H = homo_matrix.copy()
                                                                                                     

                self.homography_matrix = H.copy()

                self.ref_data = []
                self.global_poses = {0: SE3().pose}
                self.global_pose = SE3()
                self.pose_ctr = 0
                self.cur_data = None
                self.frame_pairs = []
                self.viz = True
                self.scale = 1
                self.img_id = 0

                self.no_matches = []
                self.inlier_pts = []
                self.cheirality_pts = []


                self.ref_kps_index = None
                self.ref_pose = SE3()
                self.good_points_3D = None

                # self.contour = np.array([[405,390],[615,350],[605,450],[385,480] ], dtype=np.int32)

                self.initialized = False
                self.pose_ctr = 0
                

                self.ref_len = 1
                self.method = 'learned'

                self.essentialMat = {'iter' : 3, 'thresh' : 0.3, 'prob' : 0.99}

                if os.path.exists(self.method.lower()) == False:
                        os.mkdir(self.method.lower())


        def update_global_pose(self, poss):
            self.global_pose.t += self.global_pose.R @ poss.t
            self.global_pose.R = self.global_pose.R @ poss.R

        def computepose_2D_2D(self,framepair):

                # 
                best_inlier_cnt = 0
                retval = False
                pose = SE3()
                left_kp, right_kp = framepair.getkeypts()

                E1, inliers = cv2.findEssentialMat(right_kp.copy(), left_kp.copy(),
                                                                                    self.cam_intr,
                                                                                    method = cv2.RANSAC,
                                                                                    prob = self.essentialMat['prob'], threshold = 5.0)

                inliers = np.squeeze(inliers)
                inliers_copy_og = inliers.copy()
                pts, R1, t1, _ = cv2.recoverPose(E1, right_kp.copy(), left_kp.copy(), self.cam_intr, mask = inliers_copy_og.copy())

                left_kp = left_kp.copy()[inliers_copy_og==1]
                right_kp = right_kp.copy()[inliers_copy_og==1]
                framepair.left_kp = left_kp.copy()
                framepair.right_kp = right_kp.copy()
                framepair.left_kp_og = left_kp.copy()
                framepair.right_kp_og = right_kp.copy()

                framepair.frame_index = framepair.frame_index[inliers_copy_og==1]

                self.vizualize_custom_matches(framepair.frame1.image, framepair.frame2.image, left_kp, right_kp)
                


                avgoptflow = np.sum(np.linalg.norm(left_kp-right_kp, axis = 1))/len(right_kp)
                if avgoptflow < 5:
                #     print("NO PARALLAX")
                    return retval, framepair
                
                if pts > 70:
                    pose.R = R1
                    pose.t = t1
                    retval = True
                    framepair.pose = pose
                else:
                    retval = False
                    return retval, framepair


                for loopindex in range(self.essentialMat['iter']):

                        
                        new_list = np.random.randint(0, left_kp.shape[0], (left_kp.shape[0]))
                        new_left_kp = left_kp.copy()[new_list]
                        new_right_kp = right_kp.copy()[new_list]
                        E, inliers = cv2.findEssentialMat(new_right_kp, new_left_kp,
                                                                                            self.cam_intr,
                                                                                            method = cv2.RANSAC,
                                                                                            prob = self.essentialMat['prob'], threshold = self.essentialMat['thresh'])
                        inliers = np.squeeze(inliers)
                        inliers_copy = inliers.copy()
                        left_kp_inliers = new_left_kp.copy()[inliers_copy==1]
                        right_kp_inliers = new_right_kp.copy()[inliers_copy==1]
                        avgoptflow = np.sum(np.linalg.norm(left_kp_inliers-right_kp_inliers, axis = 1))/len(left_kp_inliers)
                        if avgoptflow < 5:
                                # print("LESS THAN 5")
                                continue
                        points, R_tmp, t_tmp, _ = cv2.recoverPose(E, new_right_kp, new_left_kp, self.cam_intr, mask = inliers_copy)

                        #ADJUSTMENT STEP

                        # t_tmp[1] = 0
                        # t_tmp = t_tmp / np.linalg.norm

                        if points < 50:
                                print("LESS THAN 50", points)
                                continue
                        
                        if inliers.sum() > best_inlier_cnt and points>50:
                                retval = True
                                best_inlier_cnt = inliers.sum()
                                
                                pose.R = R_tmp
                                pose.t = t_tmp
                                
                                framepair.cheirality_pts_ct = points
                                framepair.inlier_pts_ct = best_inlier_cnt
                                framepair.pose = pose
                                framepair.ess_mat = E
                                framepair.inl = inliers


                return retval, framepair


        def computepose_3D_2D(self,framepair):

                # 
                best_inlier_cnt = 0
                retval = False
                pose = SE3()

                left_kp, right_kp = framepair.getkeypts()

                # self.vizualize_custom_matches(framepair.frame1.image, framepair.frame2.image, left_kp, right_kp, 'BEFORE ESSMAT')
                # print("PNP BETWEEN ", framepair.frame1.filename, framepair.frame2.filename, len(left_kp))

                E1, inliers = cv2.findEssentialMat(right_kp.copy(), left_kp.copy(),
                                                                                    self.cam_intr,
                                                                                    method = cv2.RANSAC,
                                                                                    prob = self.essentialMat['prob'], threshold = 3.0)

                inliers = np.squeeze(inliers)
                inliers_copy_og = inliers.copy()


                left_kp = left_kp.copy()[inliers_copy_og==1]
                right_kp = right_kp.copy()[inliers_copy_og==1]
                # print(len(left_kp), len(inliers_copy_og))
                # print(framepair.frame_index)

                # print("#################### ESS MAT INLIERS : ", len(left_kp), " OUT OF ", len(inliers))
                # self.vizualize_custom_matches(framepair.frame1.image, framepair.frame2.image, left_kp, right_kp, 'AFTER ESSMAT')

                points_3D = self.good_points_3D[framepair.frame_index[..., 0]][inliers_copy_og==1]



                framepair.left_kp = left_kp.copy()
                framepair.right_kp = right_kp.copy()
                framepair.left_kp_og = left_kp.copy()
                framepair.right_kp_og = right_kp.copy()
                framepair.frame_index = framepair.frame_index[inliers_copy_og==1]


                avgoptflow = np.sum(np.linalg.norm(left_kp-right_kp, axis = 1))/len(right_kp)
                if avgoptflow < 5 or len(left_kp) < 50:
                #     print("NO PARALLAX")
                    return retval, framepair, 1000, 0
                

                # print("DOING PNP with %d points out of %d points between %s and %s"%(len(i1), len(left_kp), framepair.frame1.filename, framepair.frame2.filename))
                best_rt = []
                best_inliers = None
                best_inlier = 0
                
                # t1 = time.time()
                for loopindex in range(self.essentialMat['iter']):
                     
                        new_list = np.random.randint(0, points_3D.shape[0], (points_3D.shape[0]))
                        new_left_kp = left_kp.copy()[new_list]
                        new_right_kp = right_kp.copy()[new_list]
                        new_points_3D = points_3D.copy()[new_list]
                        # print("$$$$$ PNP ", len(new_right_kp))
                        # flag, r, t = cv2.solvePnP(objectPoints = new_points_3D, imagePoints = new_right_kp, cameraMatrix = self.cam_intr, distCoeffs = None, rvec = cv2.Rodrigues(R1_tmp)[0], tvec = t1_tmp, useExtrinsicGuess = True, flags = cv2.SOLVEPNP_ITERATIVE)
                        # inlier = np.random.rand(60,2)
                        # print(inlier.shape[0])
                        flag, r, t, inlier = cv2.solvePnPRansac(objectPoints = new_points_3D, imagePoints = new_right_kp, cameraMatrix = self.cam_intr, distCoeffs = None, iterationsCount = 1000, reprojectionError=1.250)
                        # flag, r, t, inlier = cv2.solvePnPRansac(objectPoints = new_points_3D, imagePoints = new_right_kp, cameraMatrix = self.cam_intr, distCoeffs = np.array([0.0427517,     0.20910611,    0.00161125,    0.00081772, -0.79102898]), iterationsCount = 1000, reprojectionError=1.75)
                        # print(inlier.shape)

                        if flag and inlier.shape[0] > best_inlier and inlier.shape[0] > 25:
                                best_rt = [r, t]
                                best_inlier = inlier.shape[0]
                                best_inliers = np.squeeze(inlier).copy()

                pose = SE3()
                if len(best_rt) != 0:
                        # print()
                        retval = True
                        r, t = best_rt
                        pose.R = cv2.Rodrigues(r)[0]
                        pose.t = t
                        pose.pose = pose.inv_pose.copy()
                        # print(pose.t.T)
                        framepair.pose = pose
                else:
                    bad=True
                #     print("NO IT IS A BAD PNP")

                return retval, framepair, len(left_kp), best_inlier



        def vizualize_matches(self,framepair, imgidx, val):
                cv_kp1 = [cv2.KeyPoint(x=pt[0], y=pt[1], _size=1) for pt in framepair.left_kp]
                cv_kp2 = [cv2.KeyPoint(x=pt[0], y=pt[1], _size=1) for pt in framepair.right_kp]
                dmtches = [cv2.DMatch(_imgIdx=0,_queryIdx=i, _trainIdx=i, _distance = 0) for i in range(len(cv_kp1))]
                out_img = cv2.drawMatches(framepair.frame1.image, cv_kp1, framepair.frame2.image, cv_kp2, dmtches[::50], None, (0,255,255), -1, None, 2)
                # cv2.imwrite('outfolder/%06d_match.jpg'%imgidx, out_img)
                cv2.imshow('match', out_img)


        def vizualize_custom_matches(self,img1, img2, kps1, kps2, name = 'match_inl'):
                cv_kp1 = [cv2.KeyPoint(x=pt[0], y=pt[1], _size=1) for pt in kps1]
                cv_kp2 = [cv2.KeyPoint(x=pt[0], y=pt[1], _size=1) for pt in kps2]
                dmtches = [cv2.DMatch(_imgIdx=0,_queryIdx=i, _trainIdx=i, _distance = 0) for i in range(len(cv_kp1))]
                out_img = cv2.drawMatches(img1, cv_kp1, img2, cv_kp2, dmtches[::5], None, (0,255,255), -1, None, 2)
                cv2.imwrite('outfolder/%06d_match_inl.jpg'%self.img_id, out_img)
                # cv2.imshow(name, out_img)
                # cv2.waitKey(0)


        def vizualize_kps(self,img1, img2, kps, val, to_update = False, draw_contour = False):


              cv_kp1 = [cv2.KeyPoint(x=pt[0], y=pt[1], _size=5) for pt in kps]
              if img1!=None:
                out_img = cv2.drawKeypoints(img1.astype(np.uint8), cv_kp1, None, (0,255,255))
                cv2_imshow(out_img)
                cv2.imwrite('outfolder/%06d_d.jpg'%val, out_img)
              # if img2 != None:
              # out_img = cv2.drawKeypoints(img2.astype(np.uint8), cv_kp1, None, (0,255,255))
              out_img = cv2.circle(img2.astype(np.uint8),(kps[0][0],kps[0][1]),10, (0, 255, 255),thickness=2, lineType=8, shift=0) 
              # cv2_imshow(out_img)
              if draw_contour:
                out_img = cv2.drawContours(out_img, [self.contours], 0, (0,255,0), 3)
              # cv2_imshow(out_img)

              # out_img = cv2.putText(out_img, "Distance of midpoint from vehicle : " + str(np.squeeze(transform_points(self.midpoint_3D, self.cur_data.pose.inv_pose))), (200,50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2 )
              # out_img = cv2.putText(out_img, "Distance travelled by vehicle : " + str(np.squeeze(self.cur_data.pose.t.T)), (200,80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255),2 )
              cv2.imwrite('outfolder/t%s_c.jpg'%val, out_img)

        def update_frames_data(self, framepair):
                self.ref_data = append_to_list(self.ref_data, self.cur_data, 2)
                self.good_points_3D = transform_points(self.good_points_3D, framepair.pose.inv_pose)
        
        def update_framepairs(self, fp):
                self.frame_pairs = append_to_list(self.frame_pairs, fp, 4)




        def triangulate_new_ref_points(self, framepair):

                ref_kp, ref_desc = framepair.frame1.get_kp_desc()
                cur_kp, cur_desc = framepair.frame2.get_kp_desc()
                matches = get_matches(ref_kp, ref_desc, cur_kp, cur_desc, framepair.frame1.image.shape)
                left_kp = ref_kp[matches[:, 0], : 2].astype(np.float32)
                right_kp = cur_kp[matches[:, 1], : 2].astype(np.float32)

                diff = np.linalg.norm(left_kp- right_kp, axis=1)
                diff_mask = diff > 5
                matches = matches[diff_mask]
                left_kp = left_kp[diff_mask]
                right_kp = right_kp[diff_mask]
                
                E1, inliers = cv2.findEssentialMat(left_kp.copy(), right_kp.copy(),
                                                   self.cam_intr, method = cv2.RANSAC,
                                                   prob = self.essentialMat['prob'], threshold = 3.0)

                
                inliers = np.squeeze(inliers)
                inliers_copy_og = inliers.copy()
                left_kp = left_kp.copy()[inliers_copy_og==1]
                right_kp = right_kp.copy()[inliers_copy_og==1]
                matches = matches[inliers_copy_og==1]

                # self.vizualize_custom_matches(framepair.frame1.image, framepair.frame2.image, left_kp, right_kp, "triangulation")

                frame_3D_pts, _, _ = triangulation(left_kp, right_kp, np.eye(4), framepair.pose.inv_pose, self.cam_intr)
                non_zero_mask_tri = frame_3D_pts[...,2] > 0

                if len(non_zero_mask_tri[non_zero_mask_tri > 0]) > 50:
                        self.ref_kps_index = matches[non_zero_mask_tri]
                        self.good_points_3D = frame_3D_pts[non_zero_mask_tri]
                # print("\n -- %d points triangulated out of %d points --"%(len(self.good_points_3D), len(frame_3D_pts)))
                return len(non_zero_mask_tri[non_zero_mask_tri > 0])


        def get_point_in_other_image(self, framepair, to_update = False):

            temp_pose = framepair.frame2.pose.inv_pose
            projectedpts = cv2.projectPoints(self.midpoint_3D, cv2.Rodrigues(temp_pose[:3, :3])[0], temp_pose[:3,3], self.cam_intr, None)[0]
            self.vizualize_kps(None, self.cur_data.image, projectedpts[0].astype(np.int32), self.img_id, to_update)
        #     cv2.waitKey(0)


        def save_poses(self):
                with open('r2d2parking.pkl', 'wb') as f: pickle.dump(self.global_poses, f)


        def get_local_scale(self,framepair):
                '''
                Find out the global scale to update the 3D coordinates of the midpoint
                '''
                

                seg_map = framepair.frame1.seg_img
                H = self.homography_matrix.copy()
                
                H_inv = np.linalg.inv(H)


                corresponding_2D_pts = framepair.frame1.keypoints[self.ref_kps_index[:, 0]].astype(np.int32)[..., :2]
                seg_map_mask = seg_map[corresponding_2D_pts[..., 1], corresponding_2D_pts[..., 0]] == 255
                # done
                cnt_keypoints = corresponding_2D_pts[seg_map_mask]
                
                # print(cnt_keypoints)
                cnt_keypoints_homo = np.c_[cnt_keypoints,np.ones(len(cnt_keypoints))]

                good_road_points_3D = self.good_points_3D[seg_map_mask]
                # print(good_road_points_3D.shape)


                # Find 3D points
                cnt_keypoints_3d = []
                for pt in cnt_keypoints_homo:
                        pt = pt.reshape((3,1))
                        cnt_keypoints_3d.append(np.dot(H_inv,pt))
                
                
                cnt_keypoints_3d = np.squeeze(np.asarray(cnt_keypoints_3d))
                cnt_keypoints_3d_normalized = np.divide(cnt_keypoints_3d.T, abs(cnt_keypoints_3d[..., -1])).T

                cnt_keypoints_depth = abs(cnt_keypoints_3d_normalized[..., 0] / 100)
                cnt_keypoints_3d = unprojection_kp(cnt_keypoints, cnt_keypoints_depth, self.cam_intr)
                cnt_depth_mask = cnt_keypoints_depth < 30
                if len(cnt_keypoints_3d[cnt_depth_mask]) > 50:
                        true_scale = find_scale(good_road_points_3D[cnt_depth_mask], cnt_keypoints_3d[cnt_depth_mask])
                else:
                        return 1


                # cv2.waitKey(1)
                return true_scale


        def get_global_scale(self,framepair,midpoint):
                '''
                Find out the global scale to update the 3D coordinates of the midpoint
                '''
                

                seg_map = framepair.frame1.seg_img
                H = self.homography_matrix.copy()
                # H[1,2] *= -1
                
                H_inv = np.linalg.inv(H)
                # print(H)
                # H_inv = np.linalg.inv(self.homography_matrix)
                # print(self.good_points_3D.shape, self.ref_kps_index[:, 0].shape)
                mid_pt_homo = np.asarray([midpoint[0], midpoint[1], 1]).reshape((3,1))
                midpoint_3D = np.dot(H_inv,mid_pt_homo)
                midpoint_3D_normalized = midpoint_3D / abs(midpoint_3D[-1])
                # print("MIDPOINT 3D " , midpoint_3D, midpoint,    midpoint_3D_normalized, framepair.pose.t.T, np.linalg.norm(framepair.pose.t.T))

                depth = abs(midpoint_3D_normalized[0] / 100)

                midpoint_3D = unprojection_kp(np.asarray([midpoint]), depth, self.cam_intr)
                # print(midpoint_3D, midpoint_3D.shape)
                # done


                corresponding_2D_pts = framepair.frame1.keypoints[self.ref_kps_index[:, 0]].astype(np.int32)[..., :2]
                seg_map_mask = seg_map[corresponding_2D_pts[..., 1], corresponding_2D_pts[..., 0]] == 255
                # done
                cnt_keypoints = corresponding_2D_pts[seg_map_mask]
                
                # print(cnt_keypoints)
                cnt_keypoints_homo = np.c_[cnt_keypoints,np.ones(len(cnt_keypoints))]
                # cnt_keypoints_homo[:,0] = cnt_keypoints[:,0]
                # cnt_keypoints_homo[:,1] = cnt_keypoints[:,1]


                good_road_points_3D = self.good_points_3D[seg_map_mask]
                # print(good_road_points_3D.shape)


                # Find 3D points
                cnt_keypoints_3d = []
                for pt in cnt_keypoints_homo:
                        pt = pt.reshape((3,1))
                        cnt_keypoints_3d.append(np.dot(H_inv,pt))
                
                
                cnt_keypoints_3d = np.squeeze(np.asarray(cnt_keypoints_3d))
                cnt_keypoints_3d_normalized = np.divide(cnt_keypoints_3d.T, abs(cnt_keypoints_3d[..., -1])).T

                cnt_keypoints_depth = abs(cnt_keypoints_3d_normalized[..., 0] / 100)
                cnt_keypoints_3d = unprojection_kp(cnt_keypoints, cnt_keypoints_depth, self.cam_intr)
                cnt_depth_mask = cnt_keypoints_depth < 40
                self.vizualize_kps(None, framepair.frame1.image,cnt_keypoints[cnt_depth_mask], 0 )
                # print(cnt_keypoints_homo[::10], cnt_keypoints_3d[::10], cnt_keypoints_3d_normalized[::10])
                # print(cnt_keypoints_3d.shape, good_road_points_3D.shape)
                # if len(cnt_keypoints_3d[cnt_depth_mask]) > 50:

                # print("MIDPOINT    ", midpoint_3D, len(cnt_keypoints_3d[cnt_depth_mask]))

                if len(cnt_keypoints_3d[cnt_depth_mask]) > 30:
                        true_scale = find_scale(good_road_points_3D[cnt_depth_mask], cnt_keypoints_3d[cnt_depth_mask])
                else:
                    print("LESS")
                    # print(cnt_keypoints_3d[cnt_depth_mask], len(cnt_keypoints_3d[cnt_depth_mask]))
                    return -1

                # true_scale = find_scale(good_road_points_3D[cnt_depth_mask], cnt_keypoints_3d[cnt_depth_mask])
                # else:
                        # return 1
                # print("TRUE SCALE : ", true_scale)
                # true_scale = find_scale_depth_only(good_road_points_3D[..., 2], cnt_keypoints_depth)


                self.midpoint_3D = midpoint_3D.reshape((1,3))
                self.midpoint_3D /= true_scale
                # cv2.waitKey(1)
                return true_scale


        def process_frame(self, img, seg_img, midpoint, frame_no, imgfile):
                '''
                contours: Area within which keypoints are to be tracked
                img: OpenCV image
                midpoint: Midpoint to be tracked
                frame_no: Frame number
                '''
                #NOTE:
                #Use midpoint and contours for 2 frames i.e until we have established 2D-2D correspondencies
                #Followed by this use pure Visual Odometry for servoing


                

                # fil = self.imgfiles[0]
                #For the first frame
                if(frame_no == 0): #First frame
                        if self.method != 'OF':
                                # img = cv2.imread(fil)
                                #Converting OpenCV image into PIL format
                                #Original Image: img
                                kp, desc = extract_features_and_desc(img)                
                                frame1 = Frame(id = 0, img = img, kps = kp, desc = desc, fil = imgfile, pose = SE3(), seg_img = seg_img)
                                
                        
                        self.ref_data = append_to_list(self.ref_data, frame1)
                        self.initialized = False
                        self.parking_spot_points_found = False
                
                else: #NOTE: frame_no = imgidx
                        cur_img = img
                        imgidx = frame_no
                        cur_fil = imgfile
                        self.img_id = imgidx
                                
                        if self.method != 'OF':
                                # cur_img = cv2.imread(cur_fil)
                                # cur_kp, cur_desc = extract_features_and_desc(cur_img, cur_fil)
                                cur_kp, cur_desc = extract_features_and_desc(cur_img)
                                frame1 = self.ref_data[-1]
                                ref_kp, ref_desc = self.ref_data[-1].get_kp_desc()
                                # self.vizualize_kps(None,cur_img,cur_kp, 0 )
                                frame2 = Frame(id = imgidx, img = cur_img, kps = cur_kp, desc = cur_desc, fil = cur_fil, pose = SE3(), seg_img = seg_img)
                                self.cur_data = frame2
                                # print(frame1.kps_index)
                                matches = get_matches(ref_kp[frame1.kps_index], ref_desc[frame1.kps_index], cur_kp, cur_desc, cur_img.shape)
                                try:
                                        ref_keypoints = ref_kp[frame1.kps_index[matches[:, 0]], : 2].astype(np.float32)
                                except Exception as e:
                                        
                                        print(e, matches.shape, frame1.kps_index.shape, ref_kp.shape)

                                # ref_keypoints = ref_kp[matches[:, 0], : 2].astype(np.float32)
                                cur_keypoints = cur_kp[matches[:, 1], : 2].astype(np.float32)
                        
                                framepair = FramePair(frame1, frame2, matches, ref_keypoints, cur_keypoints, matches)

                        
                        if self.initialized == False:
                            retval, framepair = self.computepose_2D_2D(framepair)
                        

                        else:
                            retval, framepair, common_pts, best_inliers = self.computepose_3D_2D(framepair)
                            scale = np.linalg.norm(framepair.pose.t)
                        


                        # if self.img_id > 145:
                        #     imdone

                        to_update = False
                        if retval==True:
                                
                                if self.initialized == False:
                                        self.triangulate_new_ref_points(framepair)
                                        global_scale = self.get_global_scale(framepair,midpoint)
                                        # if global_scale < 0.70 or global_scale > 4.00:
                                        if global_scale < 0.50 or global_scale > 4.00:
                                                # self.get_point_in_other_image(framepair, to_update)
                                            # print("************************VERY POOR*******************", global_scale)
                                            return 
                                        
                                        framepair.frame1.kps_index = self.ref_kps_index[:, 0]
                                        framepair.frame2.kps_index = self.ref_kps_index[:, 1]
                                        
                                        # print("FOUND ABSOLUTE SCALE : ", global_scale, framepair.frame1.filename, framepair.frame2.filename, self.midpoint_3D * global_scale)
                                        scale = global_scale
                                        self.global_scale = scale
                                        # framepair.pose.t *= scale
                                        # self.good_points_3D *= scale
                                        common_pts = 250
                                        best_inliers = 100                        
                                        self.initialized=True
                                        to_update = True
                                

                                # if scale > 2.0 or common_pts < 200 :
                                if scale > 1.75 or common_pts < 200 :
                                        # print(scale, common_pts, best_inliers)
                                        # print("INSERTING KEYFRAME")
                                        no_of_triangulated = self.triangulate_new_ref_points(framepair)
                                        if no_of_triangulated < 50:
                                                # print("BAD TRIANGULATION ", no_of_triangulated)
                                                return 
                                        framepair.frame1.kps_index = self.ref_kps_index[:, 0]
                                        framepair.frame2.kps_index = self.ref_kps_index[:, 1]

                                        # scale = self.get_local_scale(framepair)     
                                        # print("LOCAL SCALE : ", scale)                            
                                        to_update = True
                                        # framepair.pose.t *= scale
                                        # self.good_points_3D *= scale

                                framepair.frame2.pose._pose = framepair.frame1.pose._pose @ framepair.pose._pose.copy()

                                self.pose_ctr += 1
                                self.global_poses[self.pose_ctr] = framepair.frame2.pose._pose
                                self.get_point_in_other_image(framepair, to_update)

                                # print("Current car pos : ", framepair.frame2.pose.t.T * self.global_scale)


                                if to_update:
                                        # print("--------------------- UPDATING REFERENCE FRAME TO %s with "%(framepair.frame2.filename), framepair.frame2.pose.t.T, common_pts)
                                        self.update_frames_data(framepair)
                        else:
                                framepair.frame2.pose._pose = framepair.frame1.pose._pose.copy()

                                # cv2.waitKey(0)
                                # cv2.destroyAllWindows()




In [27]:
import os
import numpy as np
import cv2
import glob
from tqdm import tqdm
from PIL import Image


idx = 0

numpyseed = 4123
midpoints = [(220, 327), (234, 308), (249, 299), (214, 327), (221, 307), (230, 312), (209, 309), (229, 299), (225, 290)]
H = np.asarray([[9.857253690556300, 11.420517249932578, 225.120435603151577],[5.517731384524304, 1.254203539695659, 3254.912664208281967],[0.031589485917738, 0.002401109962928, 1.000000000000000]])

cam_intr = np.asarray([[332.9648157406122, 0.0, 310.8472797033171], [0.0, 444.0950902369522, 252.76060777256825], [0.0, 0.0, 1.0]])


weights_path = 'model_final_final_iisc_idd_16kweights.pth'
auto_park_obj = auto_park_vision(weights_path)
points_ls = [[620,200,1],[620,-200,1],[1120,-200,1],[1120,200,1]]
points_2d_ls = world_2d(points_ls) #[967,427],[295,438],[438,309],[817,300]


for seq_no in range(1, 2):
    print(seq_no + 5)
    np.random.seed(numpyseed)
    images_directory = "all_sequences/%02d"%(seq_no)
    vo = VisualOdometry(cam_intr, H)
    start_no = 0
    for index, file_path in enumerate(sorted(glob.glob(images_directory + "/left*"))[start_no::5]):
        if index ==1 :
            continue
        frame = cv2.imread(file_path)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (640,480), interpolation = cv2.INTER_LANCZOS4)
        frame_pil = Image.fromarray(frame)
        midpoint_2D = midpoints[seq_no]

        seg_img = auto_park_obj.forward_pass(frame_pil,img_path=None)

        vo.process_frame(frame, seg_img, midpoint_2D, index,  file_path)
    print(vo.cur_data.pose.t.T * vo.global_scale)
      

6
[[-0.95625781 -0.79484716  6.37870119]]
